# Fish Speech

### For Windows User / win用户

In [1]:
!chcp 65001

Active code page: 65001


### For Linux User / Linux 用户

In [ ]:
import locale
locale.setlocale(locale.LC_ALL, 'en_US.UTF-8')

### Prepare Model

In [6]:
# For Chinese users, you probably want to use mirror to accelerate downloading
# !set HF_ENDPOINT=https://hf-mirror.com
# !export HF_ENDPOINT=https://hf-mirror.com 

!hf download fishaudio/openaudio-s1-mini --local-dir checkpoints/openaudio-s1-mini/

C:\Users\PC\Desktop\fish-speech\checkpoints\openaudio-s1-mini



Fetching 7 files: 100%|██████████| 7/7 [00:00<00:00, 268.64it/s]


## WebUI Inference

> You can use --compile to fuse CUDA kernels for faster inference (10x).

In [7]:
!python tools/run_webui.py \
    --llama-checkpoint-path checkpoints/openaudio-s1-mini \
    --decoder-checkpoint-path checkpoints/openaudio-s1-mini/codec.pth \
    # --compile

Traceback (most recent call last):
  File "c:\Users\PC\Desktop\fish-speech\tools\run_webui.py", line 11, in <module>
    from fish_speech.inference_engine import TTSInferenceEngine
  File "C:\Users\PC\Desktop\fish-speech\fish_speech\inference_engine\__init__.py", line 9, in <module>
    from fish_speech.inference_engine.reference_loader import ReferenceLoader
  File "C:\Users\PC\Desktop\fish-speech\fish_speech\inference_engine\reference_loader.py", line 11, in <module>
    from fish_speech.utils.file import (
  File "C:\Users\PC\Desktop\fish-speech\fish_speech\utils\__init__.py", line 4, in <module>
    from .instantiators import instantiate_callbacks, instantiate_loggers
  File "C:\Users\PC\Desktop\fish-speech\fish_speech\utils\instantiators.py", line 5, in <module>
    from pytorch_lightning import Callback
  File "C:\Users\PC\.pyenv\pyenv-win\versions\3.11.8\Lib\site-packages\pytorch_lightning\__init__.py", line 27, in <module>
    from pytorch_lightning.callbacks import Callback  #

## Break-down CLI Inference

### 1. Encode reference audio: / 从语音生成 prompt: 

You should get a `fake.npy` file.

你应该能得到一个 `fake.npy` 文件.

In [4]:
## Enter the path to the audio file here
src_audio = r"D:\PythonProject\vo_hutao_draw_appear.wav"

!python fish_speech/models/dac/inference.py \
    -i {src_audio} \
    --checkpoint-path "checkpoints/openaudio-s1-mini/codec.pth"

from IPython.display import Audio, display
audio = Audio(filename="fake.wav")
display(audio)

Traceback (most recent call last):
  File "c:\Users\PC\Desktop\fish-speech\fish_speech\models\dac\inference.py", line 17, in <module>
    from fish_speech.utils.file import AUDIO_EXTENSIONS
  File "C:\Users\PC\Desktop\fish-speech\fish_speech\utils\__init__.py", line 4, in <module>
    from .instantiators import instantiate_callbacks, instantiate_loggers
  File "C:\Users\PC\Desktop\fish-speech\fish_speech\utils\instantiators.py", line 5, in <module>
    from pytorch_lightning import Callback
  File "C:\Users\PC\.pyenv\pyenv-win\versions\3.11.8\Lib\site-packages\pytorch_lightning\__init__.py", line 27, in <module>
    from pytorch_lightning.callbacks import Callback  # noqa: E402
    ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\PC\.pyenv\pyenv-win\versions\3.11.8\Lib\site-packages\pytorch_lightning\callbacks\__init__.py", line 14, in <module>
    from pytorch_lightning.callbacks.batch_size_finder import BatchSizeFinder
  File "C:\Users\PC\.pyenv\pyenv-win\versions\3.

### 2. Generate semantic tokens from text: / 从文本生成语义 token:

> This command will create a codes_N file in the working directory, where N is an integer starting from 0.

> You may want to use `--compile` to fuse CUDA kernels for faster inference (~30 tokens/second -> ~300 tokens/second).

> 该命令会在工作目录下创建 codes_N 文件, 其中 N 是从 0 开始的整数.

> 您可以使用 `--compile` 来融合 cuda 内核以实现更快的推理 (~30 tokens/秒 -> ~300 tokens/秒)

In [5]:
!python fish_speech/models/text2semantic/inference.py \
    --text "hello world" \
    --prompt-text "The text corresponding to reference audio" \
    --prompt-tokens "fake.npy" \
    --checkpoint-path "checkpoints/openaudio-s1-mini" \
    --num-samples 2
    # --compile

2025-10-31 03:25:15.660 | INFO     | __main__:main:644 - Loading model ...
2025-10-31 03:25:15.804 | INFO     | fish_speech.models.text2semantic.llama:from_pretrained:432 - Loading model from checkpoints\openaudio-s1-mini, config: DualARModelArgs(model_type='dual_ar', vocab_size=155776, n_layer=28, n_head=16, dim=1024, intermediate_size=3072, n_local_heads=8, head_dim=128, rope_base=1000000, norm_eps=1e-06, max_seq_len=8192, dropout=0.0, tie_word_embeddings=False, attention_qkv_bias=False, attention_o_bias=False, attention_qk_norm=True, codebook_size=4096, num_codebooks=10, use_gradient_checkpointing=True, initializer_range=0.03125, is_reward_model=False, scale_codebook_embeddings=True, n_fast_layer=4, fast_dim=1024, fast_n_head=16, fast_n_local_heads=8, fast_head_dim=64, fast_intermediate_size=3072, fast_attention_qkv_bias=False, fast_attention_qk_norm=False, fast_attention_o_bias=False)
2025-10-31 03:25:21.126 | INFO     | fish_speech.models.text2semantic.llama:from_pretrained:494 - 

### 3. Generate speech from semantic tokens: / 从语义 token 生成人声:

In [ ]:
!python fish_speech/models/dac/inference.py \
    -i "codes_0.npy" \
    --checkpoint-path "checkpoints/openaudio-s1-mini/codec.pth"

from IPython.display import Audio, display
audio = Audio(filename="fake.wav")
display(audio)